# Feedbacks CERES via SW fluxes

(1) Reconstruit les données avec les PCs et EOFs des flux SW. 

(2) Multi Linéaire Régression des PCs sur les feedbacks pour trouver les coefficients Beta. 

(3) Retrouver les PCs des observations CERES. 

(4) Prédire les feedbacks des observations.

# Importe modules

In [1]:
# Computational modules 
%matplotlib inline
import xarray as xr
import glob
import os
import numpy as np
import netCDF4
from netCDF4 import Dataset
import pandas as pd
import re
from array import array
from pylab import *
#import geopandas
from eofs.xarray import Eof
from eofs.multivariate.standard import MultivariateEof

# Plotting modules 
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas.plotting
import matplotlib.ticker as ticker
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm

# Scikit-learn
from sklearn import linear_model
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.neural_network import MLPRegressor

# Functions definition

In [2]:
import sys
sys.path.append('/data/home/globc/peatier/CNRMppe')
import Fonctions
from Fonctions import get_wavg_budget_df
from Fonctions import wavg 
from Fonctions import plotlines_Xdf
from Fonctions import plotlines_1df
from Fonctions import Deltas_Lambda
from Fonctions import get_3D_budget_xarr
from Fonctions import get_3D_xarr
from Fonctions import get_3D_SW_xarr
from Fonctions import get_3D_LW_xarr

# EOF, SW_mean and PCs

In [3]:
SW_mean =  pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_SW_mean.npy').to_xarray().to_array()
SW_mean = SW_mean[0,:,:].rename({'variable':'mode'})
SW_mean

<xarray.DataArray (lat: 128, lon: 256)>
array([[126.76994 , 126.76994 , 126.76994 , ..., 126.76994 , 126.76994 ,
        126.76994 ],
       [121.13958 , 121.13958 , 121.13958 , ..., 121.13958 , 121.13958 ,
        121.13958 ],
       [124.57382 , 124.57382 , 124.57382 , ..., 124.69056 , 124.57382 ,
        124.57382 ],
       ...,
       [115.06742 , 115.06742 , 115.06742 , ..., 114.464325, 115.06742 ,
        115.06742 ],
       [115.30856 , 115.30856 , 115.30856 , ..., 115.30856 , 115.30856 ,
        115.30856 ],
       [115.79212 , 115.79212 , 115.79212 , ..., 115.79212 , 115.79212 ,
        115.79212 ]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon      (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    mode     <U7 'SW_mean'

In [4]:
SW_anom =  pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_SW_anom.npy').to_xarray().to_array()
SW_anom = SW_anom[0,:,:].rename({'variable':'mode'})
SW_anom

<xarray.DataArray (time: 104, lat: 128, lon: 256)>
array([[[ 1.0198593 ,  1.0198593 ,  1.0198593 , ...,  1.0198593 ,
          1.0198593 ,  1.0198593 ],
        [ 0.21433258,  0.21433258,  0.21433258, ...,  0.21433258,
          0.21433258,  0.21433258],
        [ 1.5374527 ,  1.5374527 ,  1.5374527 , ...,  1.343956  ,
          1.5374527 ,  1.5374527 ],
        ...,
        [-0.77801514, -0.77801514, -0.77801514, ..., -0.97159576,
         -0.77801514, -0.77801514],
        [-1.1984177 , -1.1984177 , -1.1984177 , ..., -1.1984177 ,
         -1.1984177 , -1.1984177 ],
        [-1.3507843 , -1.3507843 , -1.3507843 , ..., -1.3507843 ,
         -1.3507843 , -1.3507843 ]],

       [[ 0.6755676 ,  0.6755676 ,  0.6755676 , ...,  0.6755676 ,
          0.6755676 ,  0.6755676 ],
        [ 0.6979904 ,  0.6979904 ,  0.6979904 , ...,  0.6979904 ,
          0.6979904 ,  0.6979904 ],
        [ 1.6893997 ,  1.6893997 ,  1.6893997 , ...,  1.2689056 ,
          1.6893997 ,  1.6893997 ],
        ...,
        [ 1.5395584 ,  1.5395584 ,  1.5395584 , ...,  1.5296707 ,
          1.5395584 ,  1.5395584 ],
        [ 0.88274384,  0.88274384,  0.88274384, ...,  0.88274384,
          0.88274384,  0.88274384],
        [-1.3444061 , -1.3444061 , -1.3444061 , ..., -1.3444061 ,
         -1.3444061 , -1.3444061 ]],

       [[-1.2915192 , -1.2915192 , -1.2915192 , ..., -1.2915192 ,
         -1.2915192 , -1.2915192 ],
        [ 0.15612793,  0.15612793,  0.15612793, ...,  0.15612793,
          0.15612793,  0.15612793],
        [-1.7292557 , -1.7292557 , -1.7292557 , ..., -0.855217  ,
         -1.7292557 , -1.7292557 ],
        ...,
        [ 0.92092896,  0.92092896,  0.92092896, ...,  0.8674469 ,
          0.92092896,  0.92092896],
        [ 1.2019043 ,  1.2019043 ,  1.2019043 , ...,  1.2019043 ,
          1.2019043 ,  1.2019043 ],
        [ 1.3247528 ,  1.3247528 ,  1.3247528 , ...,  1.3247528 ,
          1.3247528 ,  1.3247528 ]],

       ...,

       [[ 0.6443634 ,  0.6443634 ,  0.6443634 , ...,  0.6443634 ,
          0.6443634 ,  0.6443634 ],
        [ 1.1606216 ,  1.1606216 ,  1.1606216 , ...,  1.1606216 ,
          1.1606216 ,  1.1606216 ],
        [ 0.89642334,  0.89642334,  0.89642334, ...,  0.9723892 ,
          0.89642334,  0.89642334],
        ...,
        [-0.31665802, -0.31665802, -0.31665802, ...,  0.02702332,
         -0.31665802, -0.31665802],
        [ 0.05947876,  0.05947876,  0.05947876, ...,  0.05947876,
          0.05947876,  0.05947876],
        [ 0.6081848 ,  0.6081848 ,  0.6081848 , ...,  0.6081848 ,
          0.6081848 ,  0.6081848 ]],

       [[-0.6146164 , -0.6146164 , -0.6146164 , ..., -0.6146164 ,
         -0.6146164 , -0.6146164 ],
        [-0.6206436 , -0.6206436 , -0.6206436 , ..., -0.6206436 ,
         -0.6206436 , -0.6206436 ],
        [ 0.37530518,  0.37530518,  0.37530518, ..., -0.55823517,
          0.37530518,  0.37530518],
        ...,
        [ 0.0246048 ,  0.0246048 ,  0.0246048 , ...,  0.4258957 ,
          0.0246048 ,  0.0246048 ],
        [ 0.44641113,  0.44641113,  0.44641113, ...,  0.44641113,
          0.44641113,  0.44641113],
        [ 0.8785858 ,  0.8785858 ,  0.8785858 , ...,  0.8785858 ,
          0.8785858 ,  0.8785858 ]],

       [[ 0.7058716 ,  0.7058716 ,  0.7058716 , ...,  0.7058716 ,
          0.7058716 ,  0.7058716 ],
        [ 0.48851013,  0.48851013,  0.48851013, ...,  0.48851013,
          0.48851013,  0.48851013],
        [-0.77703094, -0.77703094, -0.77703094, ..., -0.44139862,
         -0.77703094, -0.77703094],
        ...,
        [-0.57437897, -0.57437897, -0.57437897, ..., -0.22710419,
         -0.57437897, -0.57437897],
        [-0.4380951 , -0.4380951 , -0.4380951 , ..., -0.4380951 ,
         -0.4380951 , -0.4380951 ],
        [-0.17319489, -0.17319489, -0.17319489, ..., -0.17319489,
         -0.17319489, -0.17319489]]], dtype=float32)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 ... 95 96 97 98 99 100 101 102 103
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 

In [5]:
eof1 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap1_SW.npy').to_xarray().to_array()
eof2 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap2_SW.npy').to_xarray().to_array()
eof3 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap3_SW.npy').to_xarray().to_array()
eof4 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap4_SW.npy').to_xarray().to_array()
eof5 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap5_SW.npy').to_xarray().to_array()

eof1 = eof1[0,:,:].rename({'variable':'mode'})
eof2 = eof2[0,:,:].rename({'variable':'mode'})
eof3 = eof3[0,:,:].rename({'variable':'mode'})
eof4 = eof4[0,:,:].rename({'variable':'mode'})
eof5 = eof5[0,:,:].rename({'variable':'mode'})

eofs = [eof1,eof2,eof3,eof4,eof5]

In [6]:
eofs_combined = xr.combine_nested(eofs, concat_dim='eofs')
#eofs_combined

In [7]:
pc1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF1pc_SW.npy')
pc2 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF2pc_SW.npy')
pc3 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF3pc_SW.npy')
pc4 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF4pc_SW.npy')
pc5 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF5pc_SW.npy')

PC_df = pd.DataFrame(pc1)
PC_df['pc1'] = pc1
PC_df['pc2'] = pc2
PC_df['pc3'] = pc3
PC_df['pc4'] = pc4
PC_df['pc5'] = pc5
PC_df = PC_df.drop(labels = 0, axis=1)
#PC_df

PC_df

,pc1,pc2,pc3,pc4,pc5
0,2.047448,-1.880354,0.695999,0.411820,1.125474
1,0.089597,0.496123,0.467096,1.178996,-0.048062
2,0.116893,-1.046918,0.690331,0.520984,-0.466149
3,0.185766,-0.481640,-1.690326,1.516757,-1.070102
4,0.715678,-2.260945,0.754597,0.866218,0.442685
...,...,...,...,...,...
97,0.171422,1.157356,1.041254,0.640260,0.379373
98,-1.242337,0.038289,0.261865,-0.553909,-1.030117
99,-0.690952,0.455630,0.653965,0.037837,0.280628
100,0.174499,0.459449,1.376661,0.565957,1.431853


In [8]:
pc1_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF1pc_SW.npy')
pc2_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF2pc_SW.npy')
pc3_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF3pc_SW.npy')
pc4_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF4pc_SW.npy')
pc5_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF5pc_SW.npy')

pc1_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF1pc_SW.npy')
pc2_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF2pc_SW.npy')
pc3_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF3pc_SW.npy')
pc4_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF4pc_SW.npy')
pc5_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF5pc_SW.npy')

In [9]:
PCs = [0]*102
for i in range(0,102,1):
    tmp = PC_df.iloc[i].values
    PCs[i] = tmp

PCs

[array([ 2.0474484 , -1.8803537 ,  0.6959992 ,  0.41181964,  1.1254739 ],
       dtype=float32),
 array([ 0.08959693,  0.4961233 ,  0.46709573,  1.1789958 , -0.04806199],
       dtype=float32),
 array([ 0.11689255, -1.0469179 ,  0.690331  ,  0.52098364, -0.46614897],
       dtype=float32),
 array([ 0.18576612, -0.48164007, -1.6903259 ,  1.5167572 , -1.0701017 ],
       dtype=float32),
 array([ 0.7156776, -2.2609448,  0.7545966,  0.8662176,  0.442685 ],
       dtype=float32),
 array([-0.20384167, -0.46242234,  1.20255   ,  0.05924474, -0.01504447],
       dtype=float32),
 array([-0.18986619, -0.7870216 , -0.88251007,  0.32856923,  1.699771  ],
       dtype=float32),
 array([-1.3129662 ,  0.14457382, -0.32069314, -0.17164196, -0.28508827],
       dtype=float32),
 array([-1.2997153 , -0.45352858, -0.947408  , -0.43291602,  0.6790156 ],
       dtype=float32),
 array([-0.4209844 ,  1.4569099 , -0.9148581 ,  0.7378951 ,  0.37586838],
       dtype=float32),
 array([-0.9309928 , -0.05362776,  

# Trouver U_obs : Reconstruciton des données

X_ens = U_ens(pc) * v_ens(eof) 

Compute anomaly : X_obs = X_obs - SW_mean_ens 

X_obs = U_obs * v_ens 

U_obs = X_obs * inv(v_ens)

## Get the CMIP6 datatset

In [10]:
def load_monthly_clim(path, filename, variables) :
    
    file = xr.open_mfdataset(path+filename,combine='by_coords')
    df=file[variables].to_dataframe()
    
    # Compute Climatological Annual Cycle :
    df1=df.reset_index(level=['time', 'lat', 'lon'])
    df1['month']=pd.DatetimeIndex(df1['time']).month # time in years
    df_mean=df1.groupby(['month', 'lat', 'lon']).mean()
    
    return df_mean

In [35]:
# Download the amip dataframes and compute the radiative budget  

path = "/data/scratch/globc/peatier/CMIP6/CNRM-CM6-1/CFMIP/amip/"
filename = "*_amip_*.nc"
variables = ['rsut']
CNRM_amip = load_monthly_clim(path, filename, variables)

path = "/data/scratch/globc/peatier/CMIP6/MRI-ESM2-0/amip/"
MRI_amip = load_monthly_clim(path, filename, variables)

path = "/data/scratch/globc/peatier/CMIP6/CanESM5/amip/"
filename = "standard_*_amip_*regrid.nc"
CAN_amip = load_monthly_clim(path, filename, variables)
CAN_amip

path = "/data/scratch/globc/peatier/CMIP6/MIROC6/"
filename = "*_amip_*regrid.nc"
MIROC_amip = load_monthly_clim(path, filename, variables)
MIROC_amip

path = "/data/scratch/globc/peatier/CMIP6/HadGEM3-GC31-LL/"
filename = "standard_*_amip_*regrid.nc"
HadGEM3_amip = load_monthly_clim(path, filename, variables)
HadGEM3_amip

path = "/data/scratch/globc/peatier/CMIP6/IPSL-CM6A-LR/"
filename = "*_amip_*regrid.nc"
IPSL_amip = load_monthly_clim(path, filename, variables)
IPSL_amip

path = "/data/scratch/globc/peatier/CMIP6/BCC-CSM2-MR/"
filename = "standard_*_amip_*regrid.nc"
BCC_amip = load_monthly_clim(path, filename, variables)
BCC_amip

rsut
month lat        lon                  
1     -88.927735 0.00000    349.693420
                 1.40625    349.704193
                 2.81250    349.707214
                 4.21875    349.720245
                 5.62500    349.733337
...                                ...
12     88.927735 352.96875    0.000000
                 354.37500    0.000000
                 355.78125    0.000000
                 357.18750    0.000000
                 358.59375    0.000000

[393216 rows x 1 columns]

In [37]:
df_obs = pd.DataFrame()
df_obs['CNRM_rsut'] = CNRM_amip['rsut']
df_obs['MRI_rsut'] = MRI_amip['rsut']
df_obs['CAN_rsut'] = CAN_amip['rsut']
df_obs['MIROC_rsut'] = MIROC_amip['rsut']
df_obs['HadGEM3_rsut'] = HadGEM3_amip['rsut']
df_obs['IPSL_rsut'] = IPSL_amip['rsut']
df_obs['BCC_rsut'] = BCC_amip['rsut']
#df_obs = df_obs.drop(['rsut', 'height'], axis = 1)
df_obs

CNRM_rsut    MRI_rsut    CAN_rsut  MIROC_rsut  \
month lat        lon                                                        
1     -88.927735 0.00000    365.54425  353.633514  337.199158  319.600586   
                 1.40625    365.54425  353.816467  337.199615  319.626617   
                 2.81250    365.54425  353.964325  337.345520  319.649200   
                 4.21875    365.54425  354.110260  337.345917  319.673462   
                 5.62500    365.54425  354.021820  337.346283  319.764221   
...                               ...         ...         ...         ...   
12     88.927735 352.96875    0.00000    0.000000    0.000000    0.000000   
                 354.37500    0.00000    0.000000    0.000000    0.000000   
                 355.78125    0.00000    0.000000    0.000000    0.000000   
                 357.18750    0.00000    0.000000    0.000000    0.000000   
                 358.59375    0.00000    0.000000    0.000000    0.000000   

                            HadGEM3_rsut   IPSL_rsut    BCC_rsut  
month lat        lon                                              
1     -88.927735 0.00000      345.913635  352.984375  349.693420  
                 1.40625      345.913391  352.991455  349.704193  
                 2.81250      345.901642  353.000153  349.707214  
                 4.21875      345.907593  353.014587  349.720245  
                 5.62500      345.909241  353.031982  349.733337  
...                                  ...         ...         ...  
12     88.927735 352.96875      0.000000    0.000000    0.000000  
                 354.37500      0.000000    0.000000    0.000000  
                 355.78125      0.000000    0.000000    0.000000  
                 357.18750      0.000000    0.000000    0.000000  
                 358.59375      0.000000    0.000000    0.000000  

[393216 rows x 7 columns]

In [38]:
obs = df_obs.groupby(['lat','lon']).mean().to_xarray()
obs

<xarray.Dataset>
Dimensions:       (lat: 128, lon: 256)
Coordinates:
  * lat           (lat) float64 -88.93 -87.54 -86.14 ... 86.14 87.54 88.93
  * lon           (lon) float64 0.0 1.406 2.812 4.219 ... 355.8 357.2 358.6
Data variables:
    CNRM_rsut     (lat, lon) float32 127.789764 127.789764 ... 114.4413 114.4413
    MRI_rsut      (lat, lon) float32 125.775024 125.81565 ... 114.13129
    CAN_rsut      (lat, lon) float32 118.87984 118.87993 ... 106.75376 106.75336
    MIROC_rsut    (lat, lon) float32 115.625984 115.63485 ... 111.750885
    HadGEM3_rsut  (lat, lon) float32 121.270546 121.274185 ... 109.26334
    IPSL_rsut     (lat, lon) float32 124.951195 124.95349 ... 104.45853
    BCC_rsut      (lat, lon) float32 122.5943 122.59816 ... 112.11503 112.11196

## Compute Anomaly

In [39]:
X_obs = {}
X_obs['CNRM'] = obs['CNRM_rsut'] - SW_mean
X_obs['MRI'] = obs['MRI_rsut'] - SW_mean
X_obs['CAN'] = obs['CAN_rsut'] - SW_mean
X_obs['MIROC'] = obs['MIROC_rsut'] - SW_mean
X_obs['HadGEM3'] = obs['HadGEM3_rsut'] - SW_mean
X_obs['IPSL'] = obs['IPSL_rsut'] - SW_mean
X_obs['BCC'] = obs['BCC_rsut'] - SW_mean
X_obs

{'CNRM': <xarray.DataArray (lat: 128, lon: 256)>
 array([[ 1.0198212 ,  1.0198212 ,  1.0198212 , ...,  1.0198212 ,
          1.0198212 ,  1.0198212 ],
        [ 0.2143631 ,  0.2143631 ,  0.2143631 , ...,  0.2143631 ,
          0.2143631 ,  0.2143631 ],
        [ 1.5374985 ,  1.5374985 ,  1.5374985 , ...,  1.3439102 ,
          1.5374985 ,  1.5374985 ],
        ...,
        [-0.77804565, -0.77804565, -0.77804565, ..., -0.9715576 ,
         -0.77804565, -0.77804565],
        [-1.1984558 , -1.1984558 , -1.1984558 , ..., -1.1984558 ,
         -1.1984558 , -1.1984558 ],
        [-1.3508224 , -1.3508224 , -1.3508224 , ..., -1.3508224 ,
         -1.3508224 , -1.3508224 ]], dtype=float32)
 Coordinates:
   * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
   * lon      (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
     mode     <U7 'SW_mean', 'MRI': <xarray.DataArray (lat: 128, lon: 256)>
 array([[-0.9949188 , -0.9542923 , -0.91423035, ..., -1.0290756 

## Trouver U_obs

In [40]:
eofs_inv = np.linalg.pinv(eofs_combined.transpose())
#eofs_inv = xr.DataArray(eofs_inv, dims=['eofs','lon','lat'])
eofs_inv

array([[[ 9.95205410e-05, -2.01782605e-04,  7.19191521e-05, ...,
          6.70459063e-04,  7.32128043e-04,  6.42226310e-04],
        [ 1.58580457e-04, -4.20564204e-04,  1.85125973e-04, ...,
          1.37181836e-03,  1.51575671e-03,  1.46462384e-03],
        [ 4.28988831e-04, -8.54313985e-05,  1.50993597e-04, ...,
          6.35869161e-04,  6.71775488e-04,  4.28409519e-04],
        [-1.15789881e-03,  4.68976243e-04, -8.86348716e-05, ...,
         -9.91045847e-04, -9.62815946e-04,  2.53830862e-04],
        [ 2.44234136e-04,  1.03246944e-03, -1.64161829e-04, ...,
         -2.60828109e-03, -3.16742365e-03, -3.69101856e-03]],

       [[ 6.24684908e-05, -2.08302648e-04,  8.25459138e-05, ...,
          7.33522233e-04,  8.07532109e-04,  7.66611483e-04],
        [ 1.26472660e-04, -4.40467265e-04,  2.05794029e-04, ...,
          1.49680825e-03,  1.65431830e-03,  1.64540892e-03],
        [ 3.79437901e-04, -8.28837074e-05,  1.59478135e-04, ...,
          6.76210446e-04,  7.24672980e-04,  5.45697

In [41]:
X_CNRM_bis = X_obs['CNRM'].expand_dims(dim={'dummy':range(0,102,1)})
X_MRI_bis = X_obs['MRI'].expand_dims(dim={'dummy':range(0,102,1)})
X_CAN_bis = X_obs['CAN'].expand_dims(dim={'dummy':range(0,102,1)})
X_MIROC_bis = X_obs['MIROC'].expand_dims(dim={'dummy':range(0,102,1)})
X_HadGEM3_bis = X_obs['HadGEM3'].expand_dims(dim={'dummy':range(0,102,1)})
X_IPSL_bis = X_obs['IPSL'].expand_dims(dim={'dummy':range(0,102,1)})
X_BCC_bis = X_obs['BCC'].expand_dims(dim={'dummy':range(0,102,1)})

In [42]:
U_CNRM = np.dot(eofs_inv,X_CNRM_bis.transpose('lon','lat', 'dummy'))
U_MRI = np.dot(eofs_inv,X_MRI_bis.transpose('lon','lat', 'dummy'))
U_CAN = np.dot(eofs_inv,X_CAN_bis.transpose('lon','lat', 'dummy'))
U_MIROC = np.dot(eofs_inv,X_MIROC_bis.transpose('lon','lat', 'dummy'))
U_HadGEM3 = np.dot(eofs_inv,X_HadGEM3_bis.transpose('lon','lat', 'dummy'))
U_IPSL = np.dot(eofs_inv,X_IPSL_bis.transpose('lon','lat', 'dummy'))
U_BCC = np.dot(eofs_inv,X_BCC_bis.transpose('lon','lat', 'dummy'))


In [43]:
U_CNRM = xr.DataArray(U_CNRM, dims=['lon','eofs','lon2', 'dummy'])
U_CNRM_avg = U_CNRM[0,:,0,0].transpose()

U_MRI = xr.DataArray(U_MRI, dims=['lon','eofs','lon2', 'dummy'])
U_MRI_avg = U_MRI[0,:,0,0].transpose()

U_CAN = xr.DataArray(U_CAN, dims=['lon','eofs','lon2', 'dummy'])
U_CAN_avg = U_CAN[0,:,0,0].transpose()

U_MIROC = xr.DataArray(U_MIROC, dims=['lon','eofs','lon2', 'dummy'])
U_MIROC_avg = U_MIROC[0,:,0,0].transpose()

U_HadGEM3 = xr.DataArray(U_HadGEM3, dims=['lon','eofs','lon2', 'dummy'])
U_HadGEM3_avg = U_HadGEM3[0,:,0,0].transpose()

U_IPSL = xr.DataArray(U_IPSL, dims=['lon','eofs','lon2', 'dummy'])
U_IPSL_avg = U_IPSL[0,:,0,0].transpose()

U_BCC = xr.DataArray(U_BCC, dims=['lon','eofs','lon2', 'dummy'])
U_BCC_avg = U_BCC[0,:,0,0].transpose()

In [44]:
#Save the U_obs_avg 
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_CNRM_SW.npy', U_CNRM_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_MRI_SW.npy', U_MRI_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_CAN_SW.npy', U_CAN_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_MIROC_SW.npy', U_MIROC_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_HadGEM3_SW.npy', U_HadGEM3_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_IPSL_SW.npy', U_IPSL_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_BCC_SW.npy', U_BCC_avg)